# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # skip header
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        wr = writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [38]:
# check csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    for line in f:
        print(line)
    
        

"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"

"Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner","Mohammad","M","0","Rodriguez","277.15873","paid","Sacramento--Roseville--Arden-Arcade, CA","961","Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)","88"

"Jimi Hendrix","Mohammad","M","1","Rodriguez","239.82975","paid","Sacramento--Roseville--Arden-Arcade, CA","961","Woodstock Inprovisation","88"

"Building 429","Mohammad","M","2","Rodriguez","300.61669","paid","Sacramento--Roseville--Arden-Arcade, CA","961","Majesty (LP Version)","88"

"The B-52's","Gianna","F","0","Jones","321.54077","free","New York-Newark-Jersey City, NY-NJ-PA","107","Love Shack","38"

"Die Mooskirchner","Gianna","F","1","Jones","169.29914","free","New York-Newark-Jersey City, NY-NJ-PA","107","Frisch und g'sund","38"

"Carrie Underwood","Gianna","F","2","Jones","195.70893","free","New York-Newark-Jersey City, NY

"Weezer","Kate","F","87","Harrell","199.94077","paid","Lansing-East Lansing, MI","605","Island In The Sun","97"

"Scandinavian Music Group","Harper","M","48","Barrett","544.88771","paid","New York-Newark-Jersey City, NY-NJ-PA","404","Kun Puut Tekee SeittiÃÂ¤","42"

"Caparezza","Aleena","F","26","Kirby","310.30812","paid","Waterloo-Cedar Falls, IA","619","Fuori Dal Tunnel (Album Version)","44"

"Billy Idol","Kate","F","89","Harrell","228.33587","paid","Lansing-East Lansing, MI","605","Flesh For Fantasy","97"

"Charttraxx Karaoke","Aleena","F","27","Kirby","225.17506","paid","Waterloo-Cedar Falls, IA","619","Fireflies","44"

"Michael BublÃÂ©","Kate","F","90","Harrell","229.04118","paid","Lansing-East Lansing, MI","605","Can't Help Falling In Love (Album Version) (Studio)","97"

"Amos Lee","Harper","M","49","Barrett","178.99057","paid","New York-Newark-Jersey City, NY-NJ-PA","404","Baby I Want You","42"

"The Notorious B.I.G.","Aleena","F","28","Kirby","252.49914","paid","Waterloo-Cedar


"Rise Against","Tegan","F","35","Levine","245.96853","paid","Portland-South Portland, ME","548","Audience Of One","80"

"Watain","Sylvie","F","6","Cruz","303.25506","free","Washington-Arlington-Alexandria, DC-VA-MD-WV","484","Sworn To The Dark","10"

"Nickel Creek","Tegan","F","36","Levine","187.55873","paid","Portland-South Portland, ME","548","Scotch & Chocolate","80"

"Ricardo Arjona","Sylvie","F","7","Cruz","210.25914","free","Washington-Arlington-Alexandria, DC-VA-MD-WV","484","Como Duele (Album)","10"

"Sherwood","Tegan","F","37","Levine","122.77506","paid","Portland-South Portland, ME","548","The last to know","80"

"Gordon Lightfoot","Tegan","F","40","Levine","172.61669","paid","Portland-South Portland, ME","548","Steel Rail Blues","80"

"The Temper Trap","Tegan","F","41","Levine","192.67873","paid","Portland-South Portland, ME","548","Fader","80"

"Moby","Tegan","F","42","Levine","228.25751","paid","Portland-South Portland, ME","548","South Side","80"

"Toby Keith","Tegan","F

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [39]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [40]:
# TO-DO:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [41]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create table songplays_by_session

In [42]:
# For table songplays_by_session, session_id and item_in_session were used as a composite primary key\
# because the queries will filter by these columns and also use them to help make up a unique key.
query = "CREATE TABLE IF NOT EXISTS songplays_by_session"
query = query + """(
                    session_id int,
                    item_in_session int,
                    artist text,
                    song text,
                    length float,
                    PRIMARY KEY(session_id, item_in_session)
                )"""
try:
    session.execute(query)
except Exception as e:
    print(e)          

#### Insert data into table songplays_by_session

In [43]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for row in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songplays_by_session(session_id, item_in_session, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))

#### Do a SELECT to verify that the data have been inserted into table songplays_by_session

In [44]:
query = "SELECT artist, song, length FROM songplays_by_session WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875



### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create table songplays_sortby_item_in_session

In [52]:
# For table songplays_sortby_item_in_session, user_id and session_id were used as a composite primary key\
# because the queries will filter by these columns and also use them to help make up a unique key.\
# item_in_session was used as clustering column causes the queries sort by this column.
query = "CREATE TABLE IF NOT EXISTS songplays_sortby_item_in_session"
query = query + """(
                    user_id int,
                    session_id int,
                    item_in_session int,
                    artist text,
                    song text,
                    first_name text,
                    last_name text,
                    PRIMARY KEY((user_id, session_id), item_in_session)
                )"""
try:
    session.execute(query)
except Exception as e:
    print(e)    

### Insert data into songplays_sortby_item_in_session

In [53]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for row in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songplays_sortby_item_in_session(user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))

#### Do a SELECT to verify that the data have been inserted into table songplays_sortby_item_in_session

In [54]:
query = "SELECT artist, song, first_name, last_name, item_in_session FROM songplays_sortby_item_in_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (f'{row.artist} - {row.song} - {row.first_name} - {row.last_name}')     

Down To The Bone - Keep On Keepin' On - Sylvie - Cruz
Three Drives - Greece 2000 - Sylvie - Cruz
Sebastien Tellier - Kilometer - Sylvie - Cruz
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie - Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create table songplays

In [56]:
# For table songplays, song and user_id were used as a composite primary key\
# because the queries will filter by these columns and also use them to help make up a unique key.
query = "CREATE TABLE IF NOT EXISTS songplays"
query = query + """(
                    song text,
                    user_id int,
                    first_name text,
                    last_name text,
                    PRIMARY KEY(song, user_id)
                )"""
try:
    session.execute(query)
except Exception as e:
    print(e)    

#### Insert data into songplays

In [ ]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for row in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songplays(song, user_id, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (row[9], row[10], row[1], row[4]))

#### Do a SELECT to verify that the data have been inserted into table songplays

In [1]:
query = "SELECT first_name, last_name FROM songplays WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.first_name, row.last_name)
                    

name 'session' is not defined


NameError: name 'rows' is not defined

### Drop the tables before closing out the sessions

#### Drop table songplays_by_session

In [26]:
query = "drop table songplays_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Drop table songplays_sortby_item_in_session

In [ ]:
query = "drop table songplays_sortby_item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Drop table songplays

In [ ]:
query = "drop table songplays"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()